In [130]:
import pickle
import numpy as np
from vcdist_funcs import *
import time

file_path = '/mnt/4T-HD/qing/intermediate/feat_pickle/'
magic_thh = 0.83
N = 300

layer_feature_dist = []
sub_type = []
view_point = []
objs = ['car', 'aeroplane', 'bicycle', 'bus', 'motorbike', 'train']
# objs = ['car']
for oo in objs:
    fname = file_path + 'res_info_' + oo + '_train.pickle'
    print('loading object {0}'.format(oo))
    with open(fname, 'rb') as fh:
        l, s, v = pickle.load(fh)
        Nidx = np.random.choice(len(s), size=(N,), replace=False)
        layer_feature_dist += [l[nii].T for nii in Nidx]
        sub_type += [s[nii] for nii in Nidx]
        view_point += [v[nii] for nii in Nidx]
        
print('total number of instances {0}'.format(len(sub_type)))

'''
N=300
layer_feature_dist = layer_feature_dist[0:N]
sub_type = sub_type[0:N]
view_point = view_point[0:N]
'''


loading object car
loading object aeroplane
loading object bicycle
loading object bus
loading object motorbike
loading object train
total number of instances 1800


'\nN=300\nlayer_feature_dist = layer_feature_dist[0:N]\nsub_type = sub_type[0:N]\nview_point = view_point[0:N]\n'

In [133]:
!pwd

/home/candy/qing_voting_139/qing_clustering


In [132]:
layer_feature_b = [None for nn in range(N*len(objs))]
patch_feature_dist = []
patch_feature_b = []
pos_rec = []
patch_size = [7,7]  # h, w of patch
for nn in range(N*len(objs)):
    layer_feature_b[nn] = (layer_feature_dist[nn]<magic_thh).astype(int)
    w,h = layer_feature_b[nn].shape[1:]
    
    for ww in range(0, w-patch_size[1]+1, 3):
        for hh in range(0, h-patch_size[0]+1, 3):
            patch_feature_dist.append(layer_feature_dist[nn][:,ww:ww+patch_size[1], hh:hh+patch_size[0]])
            patch_feature_b.append(layer_feature_b[nn][:,ww:ww+patch_size[1], hh:hh+patch_size[0]])
            pos_rec.append([nn, ww, hh])
            
            
print(len(patch_feature_dist))

16200


In [127]:
pos_rec[9111]

[322, 36, 0]

In [128]:
patch_feature_dist_f = np.array([dd.reshape(-1,) for dd in patch_feature_dist])
patch_feature_dist_f_norm = np.sqrt(np.sum(patch_feature_dist_f**2, 1)).reshape(-1,1)
patch_feature_dist_f = patch_feature_dist_f/patch_feature_dist_f_norm

In [129]:
from sklearn.cluster import KMeans
n_cls=20
km = KMeans(n_clusters=n_cls, init='k-means++', random_state=99, n_jobs=-5)
assignment = km.fit_predict(patch_feature_dist_f)
centers = km.cluster_centers_
print(centers.shape)

(20, 10192)


In [122]:
print(centers.shape, assignment.shape)
print(sum(assignment==3))

(20, 10192) (11958,)
957


In [123]:
import sys
import cv2
import math
import scipy.io as sio
sys.path.insert(0, '/home/candy/qing_voting_139/qing_voting_py/')
from config_voting import *
from myresize import myresize
%load_ext autoreload
%autoreload 2

nni=0
set_type = 'train'
img_ls = [None for nn in range(N*len(objs))]
for category in objs:
    dir_img = Dataset['img_dir'].format(category)
    dir_anno = Dataset['anno_dir'].format(category)
    file_list = Dataset['{0}_list'.format(set_type)].format(category)
    assert(os.path.isfile(file_list))
    
    with open(file_list, 'r') as fh:
        content = fh.readlines()
        
    img_list = [x.strip().split() for x in content]
    img_num = N
    
    for nn in range(img_num):
        file_img = os.path.join(dir_img, '{0}.JPEG'.format(img_list[nn][0]))
        assert(os.path.isfile(file_img))
        img = cv2.imread(file_img)
        height, width, _ = img.shape
        
        file_anno = os.path.join(dir_anno, '{0}.mat'.format(img_list[nn][0]))
        assert(os.path.isfile(file_anno))
        mat_contents = sio.loadmat(file_anno)
        record = mat_contents['record']
        objects = record['objects']
        bbox = objects[0,0]['bbox'][0,int(img_list[nn][1])-1][0]
        bbox = [max(math.ceil(bbox[0]), 1), max(math.ceil(bbox[1]), 1), \
                min(math.floor(bbox[2]), width), min(math.floor(bbox[3]), height)]
        
        patch = img[bbox[1]-1: bbox[3], bbox[0]-1: bbox[2], :]
        img_ls[nni] = myresize(patch, 224, 'short')
        nni = nni+1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [124]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (20.0, 200.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

n_exp = 20
patch_size=100

big_p = np.zeros((10+(patch_size+10)*n_cls, 10+(patch_size+10)*n_exp, 3))
for pp in range(n_cls):
    idd_s = np.where(assignment==pp)[0]
    target = centers[pp].reshape(1,-1)
    dist = np.sum((patch_feature_dist_f[idd_s]-target)**2,1)
    sort_i = np.argsort(dist)
    
    for i,idd in enumerate(sort_i[0:n_exp]):
        nn, ww, hh = np.array(pos_rec)[idd_s][idd]
        patch = img_ls[nn][(hh+3)*16-42:(hh+3)*16-42+patch_size, (ww+3)*16-42:(ww+3)*16-42+patch_size, :]
        ph,pw = patch.shape[0:2]
        r_start = 10+(patch_size+10)*pp
        c_start = 10+(patch_size+10)*i
        big_p[r_start:r_start+ph, c_start:c_start+pw,:] = patch

cv2.imwrite('./test.png', big_p)
'''
f, ax = plt.subplots(30,10)
for pp in range(30):
    idd_s = np.where(assignment==pp)[0]
    target = centers[pp].reshape(1,-1)
    dist = np.sum((patch_feature_dist_f[idd_s]-target)**2,1)
    sort_i = np.argsort(dist)
    
    for i,idd in enumerate(sort_i[0:10]):
        nn, ww, hh = np.array(pos_rec)[idd_s][idd]
        patch = img_ls[nn][(hh+3)*16-42:(hh+3)*16-42+100, (ww+3)*16-42:(ww+3)*16-42+100, :]
        ax[pp][i].imshow(cv2.cvtColor(patch, cv2.COLOR_BGR2RGB))
        
plt.show()
'''

'\nf, ax = plt.subplots(30,10)\nfor pp in range(30):\n    idd_s = np.where(assignment==pp)[0]\n    target = centers[pp].reshape(1,-1)\n    dist = np.sum((patch_feature_dist_f[idd_s]-target)**2,1)\n    sort_i = np.argsort(dist)\n    \n    for i,idd in enumerate(sort_i[0:10]):\n        nn, ww, hh = np.array(pos_rec)[idd_s][idd]\n        patch = img_ls[nn][(hh+3)*16-42:(hh+3)*16-42+100, (ww+3)*16-42:(ww+3)*16-42+100, :]\n        ax[pp][i].imshow(cv2.cvtColor(patch, cv2.COLOR_BGR2RGB))\n        \nplt.show()\n'

In [37]:
idd_ls

array([ 313,  317,  411,  415,  419,  423,  430,  434,  479,  503,  614,
        618,  622,  802,  806,  810,  814,  827,  831,  835,  951,  955,
       1051, 1218, 1222, 1226, 1230, 1454, 1458, 1558, 1562, 1566, 1570,
       1583, 1638, 1987, 1991, 1995, 1999, 2003, 2007, 2031, 2035, 2039,
       2043, 2671, 2675, 2679, 2683, 2687, 2726, 2730, 2750, 2754, 2758,
       2935, 2947, 2951, 2955, 2971, 2975, 2979, 3128, 3132, 3154, 3158,
       3299, 3303, 3454, 3458, 3462, 3615, 3619, 3623, 3627, 3647, 3651,
       3655, 3659, 3674, 3798, 3802, 3806, 3810, 3814, 3870, 4010, 4070,
       4074, 4122, 4126, 4159, 4163, 4187, 4226, 4230, 4234, 4238, 4242,
       4246, 4346, 4350, 4354, 4358, 4410, 4430, 4450, 4454, 4458, 4462,
       4466, 4535, 4539, 4594])

In [39]:
pos_rec[313], pos_rec[317], pos_rec[411]

([5, 22, 2], [5, 24, 2], [7, 6, 6])

In [41]:
img_ls[5].shape

(224, 336, 3)